In [7]:
import os
import os
import markdown2
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from autogen_core.tools import FunctionTool
from autogen_core.tools import FunctionTool
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv

load_dotenv()

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-pro",  # Ensure this model supports the required features
    api_key=os.getenv("GEMINI_API_KEY"),
    api_type="google",
    model_info={
        "vision": True,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
    },
)

def generate_pdf(content: str, filename: str = "output.pdf", output_dir: str = "output") -> str:
    """Generate a PDF file from text content with proper word wrapping and markdown support."""
    
    os.makedirs(output_dir, exist_ok=True)

    # Ensure the filename has a .pdf extension
    if not filename.lower().endswith(".pdf"):
        filename += ".pdf"

    filepath = os.path.join(output_dir, filename)

    # Convert Markdown to HTML
    html_content = markdown2.markdown(content)

    # Create a PDF document
    doc = SimpleDocTemplate(filepath, pagesize=letter)
    styles = getSampleStyleSheet()
    story = []

    # Convert HTML to ReportLab Paragraph
    for paragraph in html_content.split("\n"):
        if paragraph.strip():  # Ignore empty lines
            story.append(Paragraph(paragraph, styles["Normal"]))
            story.append(Spacer(1, 12))  # Add spacing between paragraphs

    # Build the PDF
    doc.build(story)

    return filepath


pdf_generator_tool = FunctionTool(
    generate_pdf,
    name="pdf_generator",
    description="Generate a PDF document from given content and save it in the output folder with markdown support."
)

report_agent_gemini = AssistantAgent(
    name="Report_Agent",
    model_client=model_client,
    tools=[pdf_generator_tool],  # Add PDF tool here
    description="Generate a report based on a given topic",
    system_message=(
        "You are a helpful assistant. Your task is to synthesize data into a high-quality "
        "literature review and generate a PDF report. Save the report in the 'output' folder."
        "Your response should end with the word 'TERMINATE'."
    ),
)

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

termination_condition = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[report_agent_gemini],
    termination_condition=termination_condition
)

await Console(
    team.run_stream(
        task="Write a literature review on 'Advanced features on Earbuds or AirPods within the last 2 years' and save it as a PDF.",
    )
)

---------- user ----------
Write a literature review on 'Advanced features on Earbuds or AirPods within the last 2 years' and save it as a PDF.
---------- Report_Agent ----------
[FunctionCall(id='', arguments='{"filename":"literature_review","output_dir":"output","content":"## Advanced Features on Earbuds or AirPods within the last 2 years\\n\\n**Introduction:**\\n\\nThe last two years have witnessed remarkable advancements in earbud and AirPod technology, pushing the boundaries of audio quality, user experience, and integration with other devices. This literature review examines these advancements, focusing on noise cancellation, spatial audio, health monitoring, and enhanced connectivity. It also explores user reviews and market trends to assess the impact of these features.\\n\\n**Noise Cancellation:**\\n\\nAdvancements in noise cancellation technology have dramatically improved the listening experience.  Adaptive noise cancellation algorithms now dynamically adjust to varying envi

TaskResult(messages=[TextMessage(source='user', models_usage=None, content="Write a literature review on 'Advanced features on Earbuds or AirPods within the last 2 years' and save it as a PDF.", type='TextMessage'), ToolCallRequestEvent(source='Report_Agent', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=467), content=[FunctionCall(id='', arguments='{"filename":"literature_review","output_dir":"output","content":"## Advanced Features on Earbuds or AirPods within the last 2 years\\n\\n**Introduction:**\\n\\nThe last two years have witnessed remarkable advancements in earbud and AirPod technology, pushing the boundaries of audio quality, user experience, and integration with other devices. This literature review examines these advancements, focusing on noise cancellation, spatial audio, health monitoring, and enhanced connectivity. It also explores user reviews and market trends to assess the impact of these features.\\n\\n**Noise Cancellation:**\\n\\nAdvancements in noi